In [1]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder, LabelBinarizer
import tensorflow as tf
import pandas as pd
import numpy as np
import time

import load_data

### One hidden layer

In [6]:
def weight_variable(shape):
    initial=tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def fc_layer(input, size_in, size_out, name="fc"):
    with tf.name_scope(name):
        w = weight_variable([size_in, size_out])
        b = bias_variable([size_out])
        act = tf.matmul(input, w) + b
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act
    

def train_model(data, label="tissue", learning_rate=1e-2):
    LOGDIR = "/tmp/tcga/"
    N_IN = data.train.X.shape[1]
    N_OUT = data.train.y[label].shape[1]
    N_HIDDEN = int(np.mean(N_IN + N_OUT))
    
    x = tf.placeholder(tf.float32, [None, N_IN], name="x")
    y_true = tf.placeholder(tf.float32, [None, N_OUT], name="labels")
    hidden = tf.nn.relu(fc_layer(x, N_IN, N_HIDDEN), name="hidden")
    y_pred = fc_layer(hidden, N_HIDDEN, N_OUT, name="softmax")
    
    with tf.name_scope("xent"):
        xent = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                labels=y_true, logits=y_pred), name="xent")
        tf.summary.scalar("xent", xent)

    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)

    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        #tf.summary.scalar("accuracy", accuracy)

    sess = tf.Session()
    summ = tf.summary.merge_all()
    writer = tf.summary.FileWriter(LOGDIR+label)
    writer.add_graph(sess.graph)
    
    # training
    t0 = time.time()
    sess.run(tf.global_variables_initializer())

    for i in range(2001):
        batch_x, batch_y = data.train.next_batch(100)
        sess.run(train_step, feed_dict={x: batch_x, y_true: batch_y[label]})
        if i % 5 == 0:
            [train_accuracy, s] = sess.run([accuracy, summ],
                feed_dict={x: tcga.train.X, y_true: tcga.train.y[label]})
    #         [test_accuracy, s] = sess.run([accuracy, summ],
    #             feed_dict={x: tcga.test.X, y_true: tcga.test.y["tissue"]})
            writer.add_summary(s, i)
            if i % 100 == 0:
                print("step", i, "training accuracy", train_accuracy)
    print("training time:", time.time() - t0)

In [5]:
tcga = load_data.read_data_sets("./data/mRNA_PCA_0.6_variance_MinMaxScaled.csv")
train_model(tcga, label="gender")

InvalidArgumentError: You must feed a value for placeholder tensor 'labels' with dtype float and shape [?,2]
	 [[Node: labels = Placeholder[dtype=DT_FLOAT, shape=[?,2], _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: xent_1/xent/_15 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_73_xent_1/xent", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'labels', defined at:
  File "/home/molly/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/molly/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/molly/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/molly/anaconda3/lib/python3.6/site-packages/tr